In [755]:
import pandas as pd
import konfuzio_sdk
from konfuzio_sdk.data import Project
from konfuzio_sdk.tokenizer.regex import WhitespaceTokenizer as KWT
from nltk.tokenize import word_tokenize, NLTKWordTokenizer
from nltk.tokenize import WhitespaceTokenizer as WT
import os
import random
from copy import deepcopy
from PIL import Image 
from PIL import ImageDraw, ImageFont
import PIL 
import json

def get_word_bboxes(document):

    raw_document = deepcopy(document)  # get Document with no Annotations
    tokenizer = KWT()
    tk_document = tokenizer(raw_document)
    tk_document.get_images(update=True)

    return tk_document

tk_doc = get_word_bboxes(document)

def process_annotated_document(document):
    dataset_column_dictionary = {
        'annotation_id': [],
        'page_index': [],
        'line_number': [],
        'top': [],
        'bottom': [],
        'x0': [],
        'y0': [],
        'x1': [],
        'y1': [],
        'label': [],
        'label_set': [],
        'start_offset': [],
        'end_offset': [],
        'offset_string': [],
        'file_name': [],
        'file_path': []
    }

    annotations = document.get_annotations()
    file_paths_array = []
    for idx, _ in enumerate(annotations):
        
        if annotations[idx].is_correct != True:
            continue
        
        nr_bboxes_in_annotations = len(annotations[idx].bboxes)

        annotation_id = annotations[idx].id_

        for bnr in range(nr_bboxes_in_annotations):

            bbox = annotations[idx].bboxes[bnr]

            page = document.pages()[bbox['page_index']]
            line_number = bbox['line_number']
            up_factor_x = page.get_image().width / page.width  # image > page, thus up_factor_x > 1
            up_factor_y = page.get_image().height / page.height # image > page, thus up_factor_y > 1

            page_index = bbox['page_index'] + 1
            top = bbox['top']
            bottom = bbox['bottom']

            x0 = bbox['x0'] * up_factor_x
            y0 = bbox['top'] * up_factor_y
            x1 = bbox['x1'] * up_factor_x
            y1 = bbox['bottom'] * up_factor_y

            #all_annotation_parts_x0 = annotations[idx].x0 * up_factor_x
            #all_annotation_parts_y0 = annotations[idx].top * up_factor_y
            #all_annotation_parts_x1 = annotations[idx].x1 * up_factor_x
            #all_annotation_parts_y1 = annotations[idx].bottom * up_factor_y

            label = annotations[idx].label.name_clean
            label_set = annotations[idx].label_set.name_clean
    
            start_offset = bbox['start_offset']
            end_offset = bbox['end_offset']
            offset_string = bbox['offset_string']
            file_name = "page_" + str(page_index) + ".png"
            file_path = os.getcwd()  + '/' + 'data_' + str(project_id) + '/documents/' + str(annotations[idx].document.id_) + "/" + file_name

            file_paths_array.append(file_path)

            dataset_column_dictionary['annotation_id'].append(annotation_id)
            dataset_column_dictionary['page_index'].append(page_index)
            dataset_column_dictionary['line_number'].append(line_number)

            dataset_column_dictionary['top'].append(top)
            dataset_column_dictionary['bottom'].append(bottom)
            dataset_column_dictionary['x0'].append(x0)
            dataset_column_dictionary['y0'].append(y0)
            dataset_column_dictionary['x1'].append(x1)
            dataset_column_dictionary['y1'].append(y1)

            #dataset_column_dictionary['all_annotation_parts_x0'].append(all_annotation_parts_x0)
            #dataset_column_dictionary['all_annotation_parts_y0'].append(all_annotation_parts_y0)
            #dataset_column_dictionary['all_annotation_parts_x1'].append(all_annotation_parts_x1)
            #dataset_column_dictionary['all_annotation_parts_y1'].append(all_annotation_parts_y1)

            dataset_column_dictionary['label'].append(label)
            dataset_column_dictionary['label_set'].append(label_set)
            dataset_column_dictionary['start_offset'].append(start_offset)
            dataset_column_dictionary['end_offset'].append(end_offset)
            dataset_column_dictionary['offset_string'].append(offset_string)
            dataset_column_dictionary['file_name'].append(file_name)
            if os.path.exists(file_path):
                dataset_column_dictionary['file_path'].append(file_path)
            else:
                dataset_column_dictionary['file_path'].append("NA")

    return dataset_column_dictionary, file_paths_array

def process_word_bboxes_document(document):
    
    dataset_column_dictionary = {
        'annotation_id': [],
        'page_index': [],
        'line_number': [],
        'offset_string': [],
        'offset_string_original': [],
        'top': [],
        'bottom': [],
        'x0': [],
        'y0': [],
        'x1': [],
        'y1': [],
        'start_offset': [],
        'end_offset': [],
        'file_name': [],
        'file_path': []
    }

    annotations = document.get_annotations()
    file_paths_array = []
    for idx, _ in enumerate(annotations):
        nr_bboxes_in_annotations = len(annotations[idx].bboxes)

        annotation_id = annotations[idx].id_

        for bnr in range(nr_bboxes_in_annotations):

            bbox = annotations[idx].bboxes[bnr]
            line_number = bbox['line_number']

            page = document.pages()[bbox['page_index']]
            up_factor_x = page.get_image().width / page.width  # image > page, thus up_factor_x > 1
            up_factor_y = page.get_image().height / page.height # image > page, thus up_factor_y > 1

            page_index = bbox['page_index'] + 1
            top = bbox['top']
            bottom = bbox['bottom']
            x0 = bbox['x0'] * up_factor_x
            y0 = bbox['top'] * up_factor_y
            x1 = bbox['x1'] * up_factor_x
            y1 = bbox['bottom'] * up_factor_y
            start_offset = bbox['start_offset']
            end_offset = bbox['end_offset']
            offset_string = bbox['offset_string']
            file_name = "page_" + str(page_index) + ".png"
            file_path =  os.getcwd() + '/' + 'data_' + str(project_id) + '/documents/' + str(annotations[idx].document.copy_of_id) + "/" + file_name
            file_paths_array.append(file_path)

            dataset_column_dictionary['page_index'].append(page_index)
            dataset_column_dictionary['annotation_id'].append(annotation_id)
            dataset_column_dictionary['start_offset'].append(start_offset)
            dataset_column_dictionary['end_offset'].append(end_offset)
            dataset_column_dictionary['offset_string'].append(offset_string)
            dataset_column_dictionary['offset_string_original'].append(offset_string)
            dataset_column_dictionary['top'].append(top)
            dataset_column_dictionary['bottom'].append(bottom)
            dataset_column_dictionary['x0'].append(x0)
            dataset_column_dictionary['y0'].append(y0)
            dataset_column_dictionary['x1'].append(x1)
            dataset_column_dictionary['y1'].append(y1)
            dataset_column_dictionary['line_number'].append(line_number)
            dataset_column_dictionary['file_name'].append(file_name)
            if os.path.exists(file_path):
                dataset_column_dictionary['file_path'].append(file_path)
            else:
                dataset_column_dictionary['file_path'].append("NA")

    return dataset_column_dictionary, file_paths_array

In [693]:
def merge_bounding_boxes(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Merge bounding boxes based on 'annotation_id' and 'annotation_id' + 'line_number'.
    
    Parameters:
    - dataframe: DataFrame containing the 'bbox', 'annotation_id', and 'line_number' columns
    
    Returns:
    - DataFrame with the added 'merged_bbboxes' and 'merged_bbboxes_by_line' columns
    """
    
    # --- First Round: Merge based on annotation_id only ---
    
    def merge_bboxes(group):
        """
        Merge bounding boxes for a group of rows.
        """
        # Ensure that the 'bbox' column contains strings
        bboxes_str = group['bbox'].astype(str)
        
        # Extract bounding boxes from the 'bbox' column
        bboxes = bboxes_str.str.strip("[]").str.split(",", expand=True).astype(float)
        
        # Compute the merged bounding box
        merged_bbox = [
            int(bboxes[0].min()),  # x0
            int(bboxes[1].min()),  # y0
            int(bboxes[2].max()),  # x1
            int(bboxes[3].max())   # y1
        ]
        
        return merged_bbox
    
    # Group by 'annotation_id' and apply the merge_bboxes function
    merged_bboxes_dict = dataframe.groupby('annotation_id').apply(merge_bboxes).to_dict()

    # Map the merged bounding boxes to the original dataframe using 'annotation_id' as the key
    dataframe['merged_bbboxes'] = dataframe['annotation_id'].map(merged_bboxes_dict)
    
    # For rows without a related annotation, copy the bounding box from 'bbox'
    dataframe['merged_bbboxes'].fillna(dataframe['bbox'], inplace=True)
    
    # --- Second Round: Merge based on annotation_id and line_number ---
    
    # Group by 'annotation_id' and 'line_number' and apply the merge_bboxes function
    key = dataframe['annotation_id'].astype(str) + "_" + dataframe['line_number'].astype(str)
    merged_bboxes_by_line_dict = dataframe.groupby(['annotation_id', 'line_number']).apply(merge_bboxes).to_dict()

    # Convert multi-index dict keys to concatenated string keys with underscore separator
    merged_bboxes_by_line_dict = {str(k[0]) + "_" + str(k[1]): v for k, v in merged_bboxes_by_line_dict.items()}

    # Map the merged bounding boxes to a new column in the dataframe using the unique key
    dataframe['merged_bbboxes_by_line'] = key.map(merged_bboxes_by_line_dict)
    
    # For rows without a merged bounding box based on both annotation_id and line_number, copy the bounding box from 'bbox'
    dataframe['merged_bbboxes_by_line'].fillna(dataframe['bbox'], inplace=True)
    
    return dataframe


def get_dataframes_from_doc(document: konfuzio_sdk.data.Document) -> konfuzio_sdk.data.Document:
    
    doc_with_word_level_bboxes = get_word_bboxes(document)
    word_boxes_dataset_column_dictionary, _ = process_word_bboxes_document(doc_with_word_level_bboxes)
    no_label_annotations_df = pd.DataFrame(word_boxes_dataset_column_dictionary)

    dataset_column_dictionary, _ = process_annotated_document(document)
    label_annotations_df = pd.DataFrame(dataset_column_dictionary)

    return no_label_annotations_df, label_annotations_df

def transfer_annotations(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:

    no_label_annotations_df = df1.copy()
    label_annotations_df = df2.copy()

    # Initialize the new columns 'label' and 'label_set' in df1 with default values
    no_label_annotations_df['label'] = 'NO_LABEL'
    no_label_annotations_df['label_set'] = 'NO_LABEL_SET'

    # Iterate through the rows of df1 and df2 and check for matching spans
    for index1, row1 in no_label_annotations_df.iterrows():
        for index2, row2 in label_annotations_df.iterrows():
            if row1['start_offset'] >= row2['start_offset'] and row1['end_offset'] <= row2['end_offset']:
                no_label_annotations_df.at[index1, 'label'] = row2['label']
                no_label_annotations_df.at[index1, 'label_set'] = row2['label_set']
                no_label_annotations_df.at[index1, 'annotation_id'] = row2['annotation_id']
                break  # break the inner loop once a match is found
    
    full_df = no_label_annotations_df

    full_df['bbox'] = list(map(list, zip(full_df['x0'].astype(int), full_df['y0'].astype(int), full_df['x1'].astype(int), full_df['y1'].astype(int))))

    return full_df

    def process_all_documents(documents):
    """
    Process all documents and organize the results in a structured dictionary.

    Parameters:
    - documents: List of documents to be processed.

    Returns:
    - tabular_document_datasets_dict: A dictionary where the key is the document's name and 
      the value is another dictionary containing 'pages' and 'document_text'.
    """
    
    tabular_document_datasets_dict = {}

    for document in documents:
        # Get dataframes from the document
        no_label_annotations_df, label_annotations_df = get_dataframes_from_doc(document)
        
        # Transfer annotations and merge bounding boxes
        full_df = transfer_annotations(no_label_annotations_df, label_annotations_df)
        full_df = merge_bounding_boxes(full_df)
        
        # Split up full_df into chunks corresponding to the pages
        pages_list = []
        for _, group in full_df.groupby('file_name'):
            pages_list.append(group)
        
        # Store the data in the main dictionary
        tabular_document_datasets_dict[document.name] = {
            'pages': pages_list,
            'document_text': document.text
        }

    return tabular_document_datasets_dict



def save_to_json(data_dict, file_path):
    """
    Save the provided dictionary to a JSON file.

    Parameters:
    - data_dict: Dictionary to be saved. It contains lists of DataFrames.
    - file_path: Path to the file where the dictionary will be saved.
    """
    
    # Convert DataFrames to a serializable format (dictionary)
    serializable_dict = {}
    for doc_key, doc_value in data_dict.items():
        pages_list_serializable = [page_df.to_dict(orient='records') for page_df in doc_value['pages']]
        serializable_dict[doc_key] = {
            'pages': pages_list_serializable,
            'document_text': doc_value['document_text']
        }

    # Save the serializable dictionary to a JSON file
    with open(file_path, 'w') as file:
        json.dump(serializable_dict, file)

def load_from_json(file_path):
    """
    Load a dictionary from a JSON file and convert serialized data back to DataFrames.

    Parameters:
    - file_path: Path to the JSON file.

    Returns:
    - Dictionary with lists of DataFrames.
    """
    
    # Load the dictionary from the JSON file
    with open(file_path, 'r') as file:
        loaded_dict = json.load(file)

    # Convert serialized data back to DataFrames
    dataframes_dict = {}
    for doc_key, doc_value in loaded_dict.items():
        pages_list_df = [pd.DataFrame(page_data) for page_data in doc_value['pages']]
        dataframes_dict[doc_key] = {
            'pages': pages_list_df,
            'document_text': doc_value['document_text']
        }

    return dataframes_dict

In [756]:


# These functions allow saving the dictionary to a JSON file and loading it back, with proper conversions.


# Needs implementation:

# Pickle:
# Python's native serialization method.
# Allows storing of Python objects without needing to convert them to another data structure.
# Quick to save and load.
# Caution: Loading pickled data from untrusted sources can be a security risk.

# HDF5 (with Pandas):
# A hierarchical data format suitable for storing large datasets.
# Pandas has built-in support for HDF5 through the HDFStore class.
# Efficiently stores large DataFrames and supports complex querying.

# SQLite (with Pandas):
# Lightweight disk-based database.
# Pandas can save DataFrames directly to SQLite with the to_sql method and read with read_sql_query.
# Useful if you want SQL capabilities without setting up a full-fledged database server.

# Parquet or Feather:
# Columnar storage file formats.
# Efficiently store large DataFrames, and they maintain data types and support fast read/write operations.
# Integrates well with Pandas using to_parquet and read_parquet methods.

# CSV Files:
# You can save each DataFrame in tabular_document_datasets_dict as a separate CSV file in a dedicated folder for each document.
# Human-readable and can be opened with many tools, but might be slower and less space-efficient than other formats.

# Database (e.g., PostgreSQL, MySQL):
# If you're dealing with very large datasets or need to perform complex queries, storing the data in a relational database might be beneficial.
# Pandas can interface with databases using the to_sql and read_sql methods.


# Util Functions

In [731]:
from PIL import Image, ImageDraw, ImageFont

def draw_boxes(image_path: str, dataframe: pd.DataFrame, bbox_column: str) -> Image.Image:
    """
    Draw bounding boxes and their labels on the image.

    Parameters:
    - image_path: Path to the image on which bounding boxes will be drawn.
    - dataframe: DataFrame containing bounding boxes and labels.
    - bbox_column: Column name in the dataframe which contains bounding boxes.
    
    Returns:
    - Modified image with bounding boxes and labels.
    """
    
    # Open the image
    image = Image.open(image_path)
    image = image.convert('RGB')
    
    width, height = image.size

    # Initialize drawing context and font
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    dataframe = dataframe[dataframe['file_path'] == image_path]

    for _, row in dataframe.iterrows():
        # Extract bounding box and label from the dataframe
        #print(row[bbox_column])
        #print(type(row[bbox_column]))
        #box = eval(row[bbox_column])  # Convert string representation of list to actual list
        box = row[bbox_column]
        label = row['label']
        
        # Draw the bounding box on the image
        draw.rectangle(box, outline="red", width=2)
        
        # Calculate label position (slightly above the bounding box)
        if label != 'NO_LABEL':
            label_position = (box[0], box[1] - 15)  # 15 pixels above the bounding box
            draw.text(label_position, label, font=font, fill="blue")
    
    return image

# For demonstration purposes, I won't run the function here. 
# You can call this function with the appropriate parameters to visualize the bounding boxes and labels on your image.


int

# Versicherungsdokumente

In [ ]:
project_id = 2
project = Project(id_=project_id, update=True, strict_data_validation=False)
project

Project 2

In [ ]:
train_documents = project.documents
no_status_documents = project.no_status_documents
preparation_documents = project.preparation_documents
test_documents = project.test_documents

In [736]:
tabular_document_datasets_dict = process_all_documents(train_documents[0:2])

In [743]:
len(tabular_document_datasets_dict['axa_MIH408883794.pdf']['pages'])#['document_text']

3

In [752]:
axa_MIH408883794_p2 = tabular_document_datasets_dict['axa_MIH408883794.pdf']['pages'][2]

In [753]:
axa_MIH408883794_p2[axa_MIH408883794_p2['label'] != 'NO_LABEL']

,annotation_id,page_index,line_number,offset_string,offset_string_original,top,bottom,x0,y0,x1,y1,start_offset,end_offset,file_name,file_path,label,label_set,bbox,merged_bbboxes,merged_bbboxes_by_line
236,1755,3,113,AXA,AXA,61.887,70.887,690.206115,107.028711,720.287900,122.593505,3127,3130,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Vertragsdetails_Versicherungsgesellschaft,Vertragsdetails,"[690, 107, 720, 122]","[690, 107, 894, 122]","[690, 107, 894, 122]"
237,1755,3,113,Krankenversicherung,Krankenversicherung,61.887,70.887,724.949773,107.028711,868.425527,122.593505,3131,3150,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Vertragsdetails_Versicherungsgesellschaft,Vertragsdetails,"[724, 107, 868, 122]","[690, 107, 894, 122]","[690, 107, 894, 122]"
238,1755,3,113,AG,AG,61.887,70.887,873.258525,107.028711,894.867007,122.593505,3151,3153,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Vertragsdetails_Versicherungsgesellschaft,Vertragsdetails,"[873, 107, 894, 122]","[690, 107, 894, 122]","[690, 107, 894, 122]"
241,1754,3,116,4924921094,4924921094,132.380,142.380,687.440450,228.940824,791.152864,246.235040,3374,3384,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Vertragsdetails_Versicherungsnummer,Vertragsdetails,"[687, 228, 791, 246]","[687, 228, 791, 246]","[687, 228, 791, 246]"
244,1750,3,118,Tamer,Tamer,168.380,178.380,386.674450,291.200000,438.530657,308.494215,3448,3453,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Kunde_Vorname,Kunde,"[386, 291, 438, 308]","[386, 291, 438, 308]","[386, 291, 438, 308]"
245,1751,3,118,Altunakar,Altunakar,168.380,178.380,448.901898,291.200000,542.243071,308.494215,3454,3463,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Kunde_Name,Kunde,"[448, 291, 542, 308]","[448, 291, 542, 308]","[448, 291, 542, 308]"
273,1756,3,128,PREMIUM,PREMIUM,324.380,334.380,127.393415,560.989761,199.992105,578.283977,4066,4073,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Einzelprodukt_Bezeichnung,Einzelprodukt,"[127, 560, 199, 578]","[127, 560, 199, 578]","[127, 560, 199, 578]"
275,1757,3,128,"18,97","18,97",324.380,334.380,822.266588,560.989761,874.122795,578.283977,4133,4138,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Einzelprodukt_Betrag,Einzelprodukt,"[822, 560, 874, 578]","[822, 560, 874, 578]","[822, 560, 874, 578]"
276,1758,3,129,KOMFORT,KOMFORT,336.380,346.380,127.393415,581.742820,199.992105,599.037035,4148,4155,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Einzelprodukt_Bezeichnung,Einzelprodukt,"[127, 581, 199, 599]","[127, 581, 199, 599]","[127, 581, 199, 599]"
278,1759,3,129,"31,37","31,37",336.380,346.380,822.266588,581.742820,874.122795,599.037035,4215,4220,page_3.png,/Users/christopherczaban/Local_Project_Files/k...,Einzelprodukt_Betrag,Einzelprodukt,"[822, 581, 874, 599]","[822, 581, 874, 599]","[822, 581, 874, 599]"


In [757]:
save_to_json(tabular_document_datasets_dict, 'konfuzio_converted_insurance_dataset.json')


file_name = 'page_1.png'
file_path = full_df[full_df['file_name'] == file_name]['file_path'].iloc[0]
print(file_path)
bboxes = full_df[full_df['file_name'] == file_name]['bbox']

draw_boxes(file_path, full_df, 'merged_bbboxes_by_line')